<a href="https://colab.research.google.com/github/appletreeleaf/ML_Projects/blob/main/BinaryClassificationModel_Pytorch_ver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.8/218.8 kB 27.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=bbabff483c87132f88d3f60a93f077623238e12f974198b7b994bb064d34e913
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [28]:
import copy

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import wandb
from easydict import EasyDict as edict

### Load Data

In [2]:
# Read data
path = '/content/drive/MyDrive/Colab Notebooks/GBT해커톤/Data'
train_data = pd.read_csv(path + "/train.csv")

# 필요한 feature만 담기
features = ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']
X = train_data[features]
y = train_data['Y_LABEL']
print(f"null: ", X.isnull().sum())

print("shape of x: {}\nshape of y: {}".format(X.shape,y.shape))

null:  COMPONENT_ARBITRARY    0
ANONYMOUS_1            0
YEAR                   0
ANONYMOUS_2            0
AG                     0
CO                     0
CR                     0
CU                     0
FE                     0
H2O                    0
MN                     0
MO                     0
NI                     0
PQINDEX                0
TI                     0
V                      0
V40                    0
ZN                     0
dtype: int64
shape of x: (14095, 18)
shape of y: (14095,)


### Preprocessing

In [3]:
label_encoders = {}

for col in ['COMPONENT_ARBITRARY', 'YEAR']:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

<ipython-input-3-1e5a1974a3f0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col])
<ipython-input-3-1e5a1974a3f0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col])


In [ ]:
#feature scaling -> 딥러닝에선 필요 없어 보이는데
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X = sc.fit_transform(X)
# X

array([[ 0.6936547 , -0.39376339, -0.66904328, ..., -0.1065502 ,
         0.89989172, -0.96600175],
       [-0.36252581, -0.42602191,  1.85326795, ..., -0.1065502 ,
        -1.31737552,  0.11914661],
       [-0.36252581, -0.17340923,  0.33988121, ..., -0.1065502 ,
        -0.74088603, -0.33221493],
       ...,
       [ 0.6936547 , -0.3221305 , -1.42573664, ..., -0.1065502 ,
        -1.11580577,  1.33970343],
       [-0.36252581, -0.15372204, -1.17350552, ..., -0.1065502 ,
        -1.31132842,  0.11914661],
       [-0.36252581, -0.29509027, -0.41681215, ..., -0.1065502 ,
        -1.25690459,  0.04391969]])

In [7]:
X = torch.tensor(X.values, dtype=torch.float32)
y = torch.tensor(y.values, dtype=torch.float32).reshape(-1, 1)

### Define models

In [9]:

class Wide(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(len(features), len(features)*3)
        self.relu = nn.ReLU()
        self.output = nn.Linear(len(features)*3, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.hidden(x))
        x = self.sigmoid(self.output(x))
        return x

class Deep(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(len(features), len(features))
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(len(features), len(features))
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(len(features), len(features))
        self.act3 = nn.ReLU()
        self.output = nn.Linear(len(features), 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x

In [10]:
# Compare model sizes
model1 = Wide()
model2 = Deep()
print(sum([x.reshape(-1).shape[0] for x in model1.parameters()]))  # 1081
print(sum([x.reshape(-1).shape[0] for x in model2.parameters()]))  # 1045

1081
1045


In [36]:
def model_train(model, X_train, y_train, X_val, y_val):
    # loss function and optimizer
    loss_fn = nn.BCELoss()  # binary cross entropy
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    n_epochs = 500   # number of epochs to run
    batch_size = 64  # size of each batch
    batch_start = torch.arange(0, len(X_train), batch_size)

    # Hold the best model
    best_acc = - np.inf   # init to negative infinity
    best_weights = None

    for epoch in range(n_epochs):
        model.train()
        with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:
                # take a batch
                X_batch = X_train[start:start+batch_size]
                y_batch = y_train[start:start+batch_size]
                # forward pass
                y_pred = model(X_batch)
                loss = loss_fn(y_pred, y_batch)
                # backward pass
                optimizer.zero_grad()
                loss.backward()
                # update weights
                optimizer.step()
                # print progress
                acc = (y_pred.round() == y_batch).float().mean()
                bar.set_postfix(
                    loss=float(loss),
                    acc=float(acc)
                )
                losses, accs = [], []
                losses.append(loss.detach().numpy())
                accs.append(acc.detach().numpy())
                wandb.log({'train_loss': np.mean(losses)})
                wandb.log({'train_acc': np.mean(losses)})

        # evaluate accuracy at end of each epoch
        model.eval()
        y_pred = model(X_val)
        acc = (y_pred.round() == y_val).float().mean()
        acc = float(acc)
        if acc > best_acc:
            best_acc = acc
            best_weights = copy.deepcopy(model.state_dict())
    # restore model and return best accuracy
    model.load_state_dict(best_weights)
    return best_acc

### WandB Setting

In [27]:
wandb.login() # API key = bcc72b6f4c340bf778bb3310374ab78fc660206e

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [32]:
args = edict({'w_project': 'test_project_3_JY',  #goorm_project_3
              'w_entity': 'goorm_team_1',
              'learning_rate': 5e-5,
              'batch_size': {'train': 8,
                             'eval': 8,
                             'test': 8},
              'accumulate': 4,     # 32개씩
              'epochs': 1,
              'seed': 42,
              'model_name': 'skt/kogpt2-base-v2',
              'max_length': 1024})

In [30]:
wandb.init(project = args.w_project, entity = args.w_entity)

wandb: Currently logged in as: 02younge (goorm_team_1). Use `wandb login --relogin` to force relogin


In [34]:
wandb.run.name = args.model_name
wandb.config.learning_rate = args.learning_rate
wandb.config.epochs = args.epochs

### Train

In [37]:
# train-test split: Hold out the test set for final model evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True)

# define 5-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True)
cv_scores_wide = []
for train, test in kfold.split(X_train, y_train):
    # create model, train, and get accuracy
    model = Wide()
    acc = model_train(model, X_train[train], y_train[train], X_train[test], y_train[test])
    print("Accuracy (wide): %.2f" % acc)
    cv_scores_wide.append(acc)
cv_scores_deep = []
for train, test in kfold.split(X_train, y_train):
    # create model, train, and get accuracy
    model = Deep()
    acc = model_train(model, X_train[train], y_train[train], X_train[test], y_train[test])
    print("Accuracy (deep): %.2f" % acc)
    cv_scores_deep.append(acc)

# evaluate the model
wide_acc = np.mean(cv_scores_wide)
wide_std = np.std(cv_scores_wide)
deep_acc = np.mean(cv_scores_deep)
deep_std = np.std(cv_scores_deep)
print("Wide: %.2f%% (+/- %.2f%%)" % (wide_acc*100, wide_std*100))
print("Deep: %.2f%% (+/- %.2f%%)" % (deep_acc*100, deep_std*100))

# rebuild model with full set of training data
if wide_acc > deep_acc:
    print("Retrain a wide model")
    model = Wide()
else:
    print("Retrain a deep model")
    model = Deep()
acc = model_train(model, X_train, y_train, X_test, y_test)
print(f"Final model accuracy: {acc*100:.2f}%")

Accuracy (wide): 0.87
Accuracy (wide): 0.87
Accuracy (wide): 0.88
Accuracy (wide): 0.87
Accuracy (wide): 0.86
Accuracy (deep): 0.88
Accuracy (deep): 0.87
Accuracy (deep): 0.87
Accuracy (deep): 0.85
Accuracy (deep): 0.87
Wide: 86.90% (+/- 0.40%)
Deep: 86.83% (+/- 0.96%)
Retrain a wide model
Final model accuracy: 86.76%


## Test data 준비하기

In [16]:
# data 불러오기
test_data = pd.read_csv(path + '/test.csv')
# 결측값 처리
test_data = test_data.fillna(0)

# 필요한 열만 추출
test_data = test_data[features]
# 레이블 인코딩
label_encoders = {}

for col in ['COMPONENT_ARBITRARY', 'YEAR']:
    le = LabelEncoder()
    test_data[col] = le.fit_transform(test_data[col])
    label_encoders[col] = le

test_data = torch.tensor(test_data.values, dtype=torch.float32)
print(test_data.shape)

torch.Size([6041, 18])


## Predict

In [53]:
model.eval()
y_preds = []
with torch.no_grad():
    # Test out inference with 5 samples
    for i in range(len(test_data)):
        y_pred = model(test_data[i])
        y_preds.append(y_pred.detach().numpy())


In [60]:
for i in range(len(y_preds)):
    if y_preds[i] >= 0.5:
        y_preds[i] = 1
    else:
        y_preds[i] = 0

print(y_preds)

[0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 

## Submission

In [63]:
submit = pd.read_csv(path + '/sample_submission.csv')
submit['Y_LABEL'] = y_preds
submit.to_csv(path + '/submit_0910_2.csv', index=False)